4. Tokenize it --- bigram/trigram
5. Reduce the set to bigrams/trigrams that appears in at least 1% of news articles

In [1]:
import pandas as pd
import numpy as np

In [2]:
#df = pd.read_csv('../media_text/uk_media_clean_10k.csv', usecols=['pubDate', 'language', 'title', 'sourceName', 'url', 'articleId'], parse_dates=['pubDate'], infer_datetime_format=True)
df = pd.read_csv('../media_text/uk_media_clean_deduped_201x_gt1k.csv', usecols=['src_name', 'text'])
df

,text,src_name
0,fundrais prepar rise challeng power walk chall...,southern daily echo
1,powerteam secur deal windfarm staff report tue...,belfast telegraph
2,former england star peter shilton ban road adm...,essex chronicle
3,debo adewumi secretari nigerian commun waltham...,chingford guardian
4,leav court dr luci dawson got sick pay one hos...,the daily mail
5,inform maria brett took centr stage carn brea ...,cornish guardian
6,publish monday februari prepar year parti rec ...,ilkeston advertiser
7,councillor critic seabird centr bryan copland ...,east lothian courier
8,main parti clash tackl britain record billion ...,banbury guardian
9,style margin10px mickey skelton back behind st...,cambridge evening news


In [3]:
import random

def RandomSample(data, size):
    """Returns random sample for given size
       If training data set is too big, we can reduce its size
    """
    return data.ix[random.sample(data.index, size)]

In [4]:
sdf  = RandomSample(df, 50000)

In [5]:
import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
import re
import string

stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#def tokenize(text):
#    tokens = nltk.word_tokenize(text)
#    stems = stem_tokens(tokens, stemmer)
#    return stems

def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split

In [16]:
#vect = CountVectorizer(tokenizer=tokenize, stop_words='english', ngram_range=(2, 3), min_df=0.01, max_features=10000)
vect = CountVectorizer(ngram_range=(2, 3), min_df=10) 
#vect = CountVectorizer(ngram_range=(2, 3)) 
#A = vect.fit_transform([sdf.text.str.cat(sep=' ')])
vect.fit(sdf.text)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(2, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [17]:
len(vect.get_feature_names())

169217

In [12]:
#%xdel vect

In [19]:
tdf = pd.DataFrame(vect.get_feature_names())

In [21]:
tdf.columns = ['ngram']

In [22]:
tdf.to_csv('../data/uk_media_50k_23gram.csv', index=False)